# B) DJI ML - Data processing

- File B: Second file
- Candidate numbers: 45, 39, 58


## 1. Imports:

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import os
import exchange_calendars as xcals
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import zscore
from scipy.stats.mstats import winsorize
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from itertools import product
from bayes_opt import BayesianOptimization

## 1. Data cleaning

### 1.1 Loading Stock and non-stock data

In [2]:
df_non =pd.read_csv("c:\\Users\\tgsog\\OneDrive\\Desktop\\DowJones_ML_project\\DataFiles\\A) Loading indicators stock data\\coms_raw_data_fin.csv")
df =pd.read_csv("C:\\Users\\tgsog\\OneDrive\\Desktop\\DowJones_ML_project\\DataFiles\\A) Loading indicators stock data\\stock_raw_data_fin.csv")

In [3]:
df.head()

,Ticker,Date,High,Low,Close,Volume
0,^DJI,2023-03-13 00:00:00-04:00,32240.349609,31624.869141,31819.140625,429090000
1,^DJI,2023-03-14 00:00:00-04:00,32306.589844,31805.400391,32155.400391,391760000
2,^DJI,2023-03-15 00:00:00-04:00,31906.470703,31429.820312,31874.570312,420910000
3,^DJI,2023-03-16 00:00:00-04:00,32281.609375,31571.460938,32246.550781,440940000
4,^DJI,2023-03-17 00:00:00-04:00,32217.320312,31728.699219,31861.980469,712100000


In [4]:
# fixing date to become datetime and take away the timestamp:
def fix_date(df, datecol):
    df[datecol] = pd.to_datetime(df[datecol].str[0:11], errors='coerce')
    df[datecol] = df[datecol].dt.date
    return df

In [5]:
#Fixing date to become timestamp
f =fix_date(df,"Date")
df_non = fix_date(df_non,"Date")

In [6]:
#finding duplicates in all stock tickers
def duplicate_time_check(df, ticker_col):
    stocks = [x for x in df[ticker_col]]
    stock_dict={}
    for x in stocks:
        dup =df[df[ticker_col] == x].duplicated().sum()
        med = {x:dup}
        stock_dict.update(med)
        dups_count = sum([x for x in stock_dict.values()])
    if  dups_count> 0:
        print(f"There are duplicates in time, {dups_count}")
    else:
        print(f"There are no duplicates {dups_count}")

In [7]:
#check duplicates
print(duplicate_time_check(df,"Ticker"), duplicate_time_check(df_non,"Ticker"))

There are no duplicates 0
There are no duplicates 0
None None


In [8]:
#Pivot and rename coloumns function
def pivot_n_rename(df, index_column, stock_column):
    df = df.pivot(index = index_column, columns = stock_column)
    df.columns = ["_".join(col) for col in df.columns]
    return df

In [9]:
#pivot and rename
df = pivot_n_rename(df,"Date","Ticker")
df_non = pivot_n_rename(df_non,"Date","Ticker")

In [10]:
#merge dataframes, and get back the Date column
df = pd.merge(df, df_non, on="Date", how="left").reset_index()

In [11]:
#check - is there any NAs?
def is_ag_na(df):
    na_ag = df.isna().sum().sum()
    if na_ag == 0:
        print(f"No NAs {na_ag}")
    else:
        print(f"There are NAs {na_ag}")

In [12]:
#Checking NAs
is_ag_na(df)

No NAs 0


In [13]:
df_sent =pd.read_csv("c:\\Users\\tgsog\\OneDrive\\Desktop\\DowJones_ML_project\\Sentiment data raw\\SentimentScore_Index_2020_2025_data_raw.csv")
colnames_sent =["Date", "sentiment"]
df_sent.columns = colnames_sent
df_sent

,Date,sentiment
0,2020-03-12,0.816822
1,2020-03-13,0.816822
2,2020-03-16,0.816822
3,2020-03-17,0.816822
4,2020-03-18,0.816822
...,...,...
1251,2025-03-05,1.000000
1252,2025-03-06,0.243465
1253,2025-03-07,0.996709
1254,2025-03-10,1.000000


In [14]:
def same_d(df1,df2, df_col):
    df1[df_col] = df1[df_col].astype(str)
    df2[df_col] = df2[df_col].astype(str)
    df = df1.merge(df2, on=df_col, how="left")
    return df


In [15]:
df = same_d(df,df_sent,"Date")

In [16]:
def convert_to_trading_days(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column])
    nyse_calendar = xcals.get_calendar("XNYS")

    start_date = df[date_column].min()
    end_date = df[date_column].max()
    date_range = pd.date_range(start=start_date, end=end_date, freq="D")

    trading_days = pd.DatetimeIndex(date_range[date_range.map(nyse_calendar.is_session)])

    df_trad = df[df[date_column].isin(trading_days)].copy()

    print(f"We filtered out the non-trading day rows from {len(df)} to {len(df_trad)}")

    df = df_trad
    return df

In [17]:
df =convert_to_trading_days(df, "Date")

We filtered out the non-trading day rows from 500 to 500


In [18]:
#Date = index 
df.index =df.Date
df =df.drop(columns = ["Date"])

### Function to make all values returns

In [19]:
def convert_all_prices_to_returns(df, price_prefixes=["Close", "High", "Low"]):
    df_returns = df.copy()
    price_columns = [col for col in df.columns if any(col.startswith(prefix) for prefix in price_prefixes)]

    for col in price_columns:
        df_returns[col] = df_returns[col].pct_change() 
        zero_mask = (df_returns[col] == 0) & (df_returns[col].shift(1) == 0)
        df_returns.loc[zero_mask, col] = np.nan 
        df_returns =df_returns.iloc[1:,:]
    return df_returns

df= convert_all_prices_to_returns(df)


In [20]:
#Inspect and interpolate the data
df = df.interpolate()

In [21]:
df["Close_^DJI"] =df["Close_^DJI"].shift(-1)
df =df.dropna()
df

,High_AAPL,High_AMGN,High_AMZN,High_AXP,High_BA,High_CAT,High_CRM,High_CSCO,High_CVX,High_DIS,...,Close_USDJPY=X,Close_USDMXN=X,Close_ZC=F,Close_ZW=F,Close_^FVX,Close_^IRX,Close_^TNX,Close_^TYX,Close_^VIX,sentiment
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-28,0.002150,0.003878,-0.011121,0.008874,0.024006,0.002619,-0.003050,-0.003721,0.011739,-0.006267,...,-0.007481,0.002614,-0.008782,0.017782,-0.028779,0.000380,-0.012076,-0.001985,0.000000,1.000000
2023-11-29,0.005286,0.005664,0.006368,0.016508,0.000845,0.010889,0.026471,0.005188,-0.000205,-0.019971,...,-0.009357,-0.002191,-0.003876,0.022529,-0.015632,-0.003802,-0.014991,-0.016136,0.022853,-1.000000
2023-11-30,-0.009214,0.009138,-0.010839,0.014521,0.035602,-0.000318,0.090713,0.001239,-0.003277,0.006649,...,-0.000163,0.009450,0.026681,0.025629,0.018725,-0.000382,0.018965,0.013929,-0.004622,0.091909
2023-12-01,0.006515,0.007503,0.002178,0.018519,0.010215,0.023618,0.043287,0.005773,0.003425,-0.010867,...,0.006529,0.005410,0.005956,0.011837,-0.033039,-0.004391,-0.028952,-0.021050,-0.022446,-1.000000
2023-12-04,-0.007883,0.003082,-0.012903,0.005564,-0.001869,-0.007419,-0.031545,-0.012300,-0.005529,-0.004632,...,-0.010851,-0.010793,-0.009150,0.032496,0.019730,0.001534,0.014671,0.004527,0.035629,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-03,0.008014,0.012507,0.006537,0.007938,0.011911,0.009423,0.003609,0.011222,0.008949,0.013685,...,0.005060,0.002779,-0.029217,-0.009311,-0.009186,0.000000,-0.012054,-0.011515,0.160469,1.000000
2025-03-04,-0.016227,0.023489,-0.033690,-0.038522,-0.056166,-0.046618,-0.037681,-0.014951,-0.037104,-0.028559,...,-0.009723,0.010298,-0.009654,-0.025376,0.003007,0.000477,0.007177,0.011873,0.032046,1.000000
2025-03-05,-0.014662,-0.002752,0.015377,-0.019365,-0.024053,0.022867,0.003532,0.004225,-0.018034,-0.011670,...,0.003309,-0.009418,0.009748,0.023626,0.013240,0.002384,0.013064,0.008855,-0.067205,1.000000


In [22]:
base_path="C:\\Users\\tgsog\\OneDrive\\Desktop\\DowJones_ML_project\\DataFiles\\B) cleaned stock data\\"
df.to_csv(base_path+"stock_clean_data_fin.csv", index=False)